### 在agent与tool之间共享记忆
- 自定义一个工具用来LLMChain总结内容
- 使用readonly memory来共享记忆
- 观察共享与不共享的区别

In [15]:
from langchain.agents import initialize_agent,Tool,AgentType
from langchain.memory import ConversationBufferMemory,ReadOnlySharedMemory
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate,MessagesPlaceholder
from langchain.utilities import SerpAPIWrapper

In [17]:
llm=ChatTongyi(model="qwen-plus",temperature=0,max_token=1000)

template="""以下是一段AI机器人和人类的对话：\n{chat_history}\n根据输入和上面的对话记录写一份对话总结。输入：{input}\n"""

prompt=PromptTemplate(
    input_variables=["input","chat_history"],
    template=template
)

memory=ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

readonly_memory=ReadOnlySharedMemory(memory=memory)

summary_chain=LLMChain(
    llm=llm,
    prompt=prompt,
    memory=readonly_memory,
    verbose=True,
)

In [19]:
# 构建工具

# 搜索工具
search=SerpAPIWrapper()

# 总结工具
def SummaryChainFunc(history):
    print("\n----------总结链开始运行----------\n")
    print("输入历史:",history)
    summary_chain.run(history)

tools=[
    Tool(
        name="Search",
        func=search.run,
        description="当需要了解实时的信息或者你不知道的事的时候可以使用搜索工具"
    ),
    Tool(
        name="Summary",
        func=SummaryChainFunc,
        description="当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用"
    )
]

print(tools)

[Tool(name='Search', description='当需要了解实时的信息或者你不知道的事的时候可以使用搜索工具', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='99f1173dd92d5d1ad2bb2cc7500fe5ef968bc070b5302aa2adec81fa67f34dd1', aiosession=None)>), Tool(name='Summary', description='当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用', func=<function SummaryChainFunc at 0x000001C9C9A72320>)]


In [8]:
# 创建agent

agent_chain=initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

print(agent_chain.agent.llm_chain.prompt.template) # 无历史记录

Answer the following questions as best you can. You have access to the following tools:

Search(query: str, **kwargs: Any) -> str - 当需要了解实时的信息或者你不知道的事的时候可以使用搜索工具
Summary(history) - 当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Summary]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [21]:
prefix="""Have a conversation with a human, answering the following questions as best as you can.You have access to the following tools:"""
suffix="""
Begin!
{chat_history}
Question: {input}
Thought:{agent_scratchpad}
"""

agent_chain=initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs={
        "prefix":prefix,
        "suffix":suffix,
        "agent_kwargs":MessagesPlaceholder("agent_scratchpad"),
        "chat_history":MessagesPlaceholder("chat_history"),
        "input":MessagesPlaceholder("input")
    },
    memory=memory
)

print(agent_chain.agent.llm_chain.prompt.template)

Have a conversation with a human, answering the following questions as best as you can.You have access to the following tools:

Search(query: str, **kwargs: Any) -> str - 当需要了解实时的信息或者你不知道的事的时候可以使用搜索工具
Summary(history) - 当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Summary]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


Begin!
{chat_history}
Question: {input}
Thought:{agent_scratchpad}



In [24]:
agent_chain.run(input="现任美国总统是谁？简单描述下他。")



> Entering new AgentExecutor chain...
我需要查找实时信息来回答这个问题。
Action: Search
Action Input: 现任美国总统是谁？简单描述下他
Observation: ['美利坚合众国总统（英语：President of the United States，缩写为POTUS；中文简称美国总统）是美国的国家元首、政府首脑和三军统帅。根据1787年通过的《美国宪法》而设立， ...', '1789年，选举人团投票一致通过，选举出乔治·华盛顿为第1任美国总统，他在其八年总统任期内没有加入任何政治派别或政党。他至今仍是唯一一位从未加入过任何政党的美国总统。第7 ...', '美国总统（President of the United States of America）是美国国家元首、政府首脑兼武装部队总司令。总统通过间接选举产生，现任总统为共和党籍的唐纳德·特朗普。', '美国前总统唐纳德·特朗普（Donald Trump）击败现任副总统卡玛拉·哈里斯（Kamala Harris），赢得了一个新的为期四年的总统任期。', '美国当选总统唐纳德·特朗普有望赢得普选，临时计票显示他领先民主党对手卡玛拉·哈里斯近500 万张选票。', '唐纳德·特朗普（Donald Trump），1946年6月14日出生于美国纽约，祖籍德国巴伐利亚自由州，毕业于宾夕法尼亚大学沃顿商学院，德裔美国共和党籍政治家、企业家、房地产商人、 ...', '美国东部时间6月27日晚9时，在佐治亚州亚特兰大市，2024年两位总统候选人——民主党现任总统拜登（Joe Biden）和共和党前总统特朗普（Donald Trump），举行首场 ...', '共和党总统候选人特朗普确定当选新任美国总统，民主党的卡玛拉·哈里斯表示将协助他顺利交接。中国国家主席习近平已向特朗普致贺电，并主张中美关系“合则 ...', '特朗普与拜登很有可能将再次在总统大选中对垒，对美国来说，这将超越自由与保守的两党二元对决，而更是关乎种族、文化、经济以及美国人认同感的决战。', '随着2024年美国总统大选日益临近，美国的富豪们也投入战局，对自己支持的政党倾囊相助。 随着泰勒·斯威夫特宣布将在大选投票中支持现任副总统卡马拉· ...']
Thought:根据搜索结果，现任美国

'现任美国总统是乔·拜登。他是美国民主党籍资深政治家，出生于1942年11月20日。拜登于2021年1月20日正式宣誓就职为第46任美国总统，任期至2025年1月20日。'

In [25]:
agent_chain.run(input="他的夫人是谁？简单描述下她。")



> Entering new AgentExecutor chain...
 我需要查找关于乔·拜登夫人的信息。
Action: Search
Action Input: 乔·拜登的夫人是谁？她的简介。
Observation: 吉尔·特蕾西·拜登（英语：Jill Tracy Biden，1951年6月3日—），婚前原姓雅各布斯（Jacobs），曾用姓史蒂文森（Stevenson），是一位美国教师，曾任美国第一夫人，即美国第46任总统乔·拜登之妻子。 吉尔·拜登生于新泽西州大西洋县的哈默顿，在宾夕法尼亚州蒙哥马利县的柳林长大。
Thought:I now know the final answer.  
Final Answer: 乔·拜登的夫人是吉尔·特蕾西·拜登（Jill Tracy Biden），她是一位美国教师，出生于1951年6月3日，曾在宾夕法尼亚州蒙哥马利县的柳林长大。作为美国第一夫人，她支持丈夫乔·拜登担任美国总统期间的各项活动，并继续致力于教育事业。

> Finished chain.


'乔·拜登的夫人是吉尔·特蕾西·拜登（Jill Tracy Biden），她是一位美国教师，出生于1951年6月3日，曾在宾夕法尼亚州蒙哥马利县的柳林长大。作为美国第一夫人，她支持丈夫乔·拜登担任美国总统期间的各项活动，并继续致力于教育事业。'

In [26]:
print(agent_chain.memory.buffer)

[HumanMessage(content='中国现任国家领导人是谁？简单描述下他。', additional_kwargs={}, response_metadata={}), AIMessage(content='中国现任国家领导人是习近平。他是中国共产党中央委员会总书记、中华人民共和国主席、中国人民解放军中央军事委员会主席、中央全面深化改革委员会主任、中央国家安全委员会主席、中央网络安全和信息化委员会主任、中央财经委员会主任、中央外事工作委员会主任、中央审计委员会主任、中央依法治国委员会主任、中央全面从严治党领导小组组长、中央机构编制委员会主任、中央教育工作领导小组组长、中央和国家机关党的建设工作领导小组组长、中央精神文明建设指导委员会主任、中央人才工作协调小组组长、中央港澳工作领导小组组长、中央对台工作领导小组组长、中央军民融合发展委员会主任、中央全面依法治国委员会主任、中央全面深化改革委员会主任、中央审计委员会主任、中央生态环境保护督察工作领导小组组长、中央全面推动长江经济带发展领导小组组长、中央黄河流域生态保护和高质量发展领导小组组长、中央乡村振兴工作领导小组组长、中央财经委员会主任、中央外事工作委员会主任、中央国家安全委员会主席、中央网络和信息化委员会主任、中央全面从严治党领导小组组长、中央机构编制委员会主任等职务。作为党和国家的核心，他领导着中国的发展方向，致力于实现中华民族伟大复兴的中国梦。', additional_kwargs={}, response_metadata={}), HumanMessage(content='现任美国总统是谁？简单描述下他。', additional_kwargs={}, response_metadata={}), AIMessage(content='现任美国总统是乔·拜登。他是美国民主党籍资深政治家，出生于1942年11月20日。拜登于2021年1月20日正式宣誓就职为第46任美国总统，任期至2025年1月20日。', additional_kwargs={}, response_metadata={}), HumanMessage(content='他的夫人是谁？简单描述下她。', additional_kwargs={}, response_metadata={}), AIMessage(content

In [27]:
agent_chain.run(input="截至目前我们都聊了什么？")



> Entering new AgentExecutor chain...
我需要总结之前的对话内容。
Action: Summary
Action Input: 中国现任国家领导人是习近平。他是中国共产党中央委员会总书记、中华人民共和国主席、中国人民解放军中央军事委员会主席等多个职务。作为党和国家的核心，他领导着中国的发展方向，致力于实现中华民族伟大复兴的中国梦。现任美国总统是乔·拜登。他是美国民主党籍资深政治家，出生于1942年11月20日。拜登于2021年1月20日正式宣誓就职为第46任美国总统，任期至2025年1月20日。乔·拜登的夫人是吉尔·特蕾西·拜登（Jill Tracy Biden），她是一位美国教师，出生于1951年6月3日，曾在宾夕法尼亚州蒙哥马利县的柳林长大。作为美国第一夫人，她支持丈夫乔·拜登担任美国总统期间的各项活动，并继续致力于教育事业。
----------总结链开始运行----------

输入历史: 中国现任国家领导人是习近平。他是中国共产党中央委员会总书记、中华人民共和国主席、中国人民解放军中央军事委员会主席等多个职务。作为党和国家的核心，他领导着中国的发展方向，致力于实现中华民族伟大复兴的中国梦。现任美国总统是乔·拜登。他是美国民主党籍资深政治家，出生于1942年11月20日。拜登于2021年1月20日正式宣誓就职为第46任美国总统，任期至2025年1月20日。乔·拜登的夫人是吉尔·特蕾西·拜登（Jill Tracy Biden），她是一位美国教师，出生于1951年6月3日，曾在宾夕法尼亚州蒙哥马利县的柳林长大。作为美国第一夫人，她支持丈夫乔·拜登担任美国总统期间的各项活动，并继续致力于教育事业。


> Entering new LLMChain chain...
Prompt after formatting:
以下是一段AI机器人和人类的对话：
[HumanMessage(content='中国现任国家领导人是谁？简单描述下他。', additional_kwargs={}, response_metadata={}), AIMessage(content='中国现任国家领导人是习近平。他是中国共产党中央委员会总书记、中华人民共和国主席、中国人民解放军中央军事委员会主席、中央全面深化改革委员会主任、中央国家安全委员会

'我们聊了中国现任国家领导人习近平，他是党和国家的核心，致力于实现中华民族伟大复兴的中国梦。还聊了现任美国总统乔·拜登及其夫人吉尔·特蕾西·拜登，吉尔作为美国第一夫人支持丈夫的各项活动，并继续致力于教育事业。'